In [0]:
%sql 

CREATE OR REPLACE TEMP VIEW encounters_hosa
USING parquet
OPTIONS (
  path="/mnt/bronze/hosa/encounters"
);


CREATE OR REPLACE TEMP VIEW encounters_hosb
USING parquet
OPTIONS (
  path="/mnt/bronze/hosb/encounters"
);

CREATE OR REPLACE TEMP VIEW encounters AS
SELECT * FROM encounters_hosa
UNION ALL 
SELECT * FROM encounters_hosb;

SELECT * FROM encounters;


In [0]:
%sql 

CREATE OR REPLACE TEMP VIEW quality_checks AS
SELECT 
concat( EncounterID,'-', datasource) as EncounterID,
EncounterID SRC_EncounterID,
PatientID,
EncounterDate,
EncounterType,
ProviderID,
DepartmentID,
ProcedureCode,
InsertDate as SRC_InsertedDate,
ModifiedDate as SRC_ModifiedDate,
datasource,
    CASE 
        WHEN EncounterID IS NULL OR PatientID IS NULL THEN TRUE
        ELSE FALSE
    END AS is_bad_data
FROM encounters;

SELECT * FROM quality_checks where datasource = 'hos-b'


In [0]:
%sql 

CREATE TABLE IF NOT EXISTS silver.encounters (
  EncounterID string,
  SRC_EncounterID string,
  PatientID string,
  EncounterDate date,
  EncounterType string,
  ProviderID string,
  DepartmentID string,
  ProcedureCode integer,
  SRC_InsertedDate date,
  SRC_ModifiedDate date,
  datasource string,
  is_bad_data boolean,
  audit_insertdate timestamp,
  audit_modifieddate timestamp,
  is_current boolean
  )
USING DELTA;


In [0]:
%sql
-- Update old record to implement SCD Type 2
MERGE INTO silver.encounters AS target
USING quality_checks AS source
ON target.EncounterID = source.EncounterID 
AND target.is_current = true
WHEN MATCHED AND (
    target.SRC_EncounterID != source.SRC_EncounterID OR
    target.PatientID != source.PatientID OR
    target.EncounterDate != source.EncounterDate OR
    target.EncounterType != source.EncounterType OR
    target.ProviderID != source.ProviderID OR
    target.DepartmentID != source.DepartmentID OR
    target.ProcedureCode != source.ProcedureCode OR
    target.SRC_InsertedDate != source.SRC_InsertedDate OR
    target.SRC_ModifiedDate != source.SRC_ModifiedDate OR
    target.datasource != source.datasource OR
    target.is_bad_data != source.is_bad_data
) THEN
  UPDATE SET
    target.is_current = false,
    target.audit_modifieddate = current_timestamp()


In [0]:
%sql
-- Insert new record to implement SCD Type 2
MERGE INTO silver.encounters AS target USING quality_checks AS source ON target.EncounterID = source.EncounterID
AND target.is_current = true
WHEN NOT MATCHED THEN
INSERT
  (
    EncounterID,
    SRC_EncounterID,
    PatientID,
    EncounterDate,
    EncounterType,
    ProviderID,
    DepartmentID,
    ProcedureCode,
    SRC_InsertedDate,
    SRC_ModifiedDate,
    datasource,
    is_bad_data,
    audit_insertdate,
    audit_modifieddate,
    is_current
  )
VALUES
  (
    source.EncounterID,
    source.SRC_EncounterID,
    source.PatientID,
    source.EncounterDate,
    source.EncounterType,
    source.ProviderID,
    source.DepartmentID,
    source.ProcedureCode,
    source.SRC_InsertedDate,
    source.SRC_ModifiedDate,
    source.datasource,
    source.is_bad_data,
    current_timestamp(),
    current_timestamp(),
    true
  );